# Airport TRaffic Visualization

Lets import libraries:

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import geopandas as gpd
import geoplot as gplt
import matplotlib.pyplot as plt
import mapclassify
import geoplot.crs as gcrs

Then read the data file:

In [ ]:
Airport_Filename="../input/covid19s-impact-on-airport-traffic/covid_impact_on_airport_traffic.csv"
Airport_Data=pd.read_csv(Airport_Filename,parse_dates=['Date'])
Airport_Data.head()

In [ ]:
Airport_Data.info()

There are no null records, good

## Preprocessing

> Lets first filter continous cities in US

In [ ]:
Airport_Data=Airport_Data[(Airport_Data.Country=='United States of America (the)') & (Airport_Data.City!='Urban Honolulu')]

Lets process the longtitude and lattitute from Centroid:

In [ ]:
Airport_Data['Longitude']=Airport_Data.Centroid.str[6:-1].str.split(' ').map(lambda x:x[0]).astype(float)
Airport_Data['Latitude']=Airport_Data.Centroid.str[6:-1].str.split(' ').map(lambda x:x[1]).astype(float)
Airport_Data=gpd.GeoDataFrame(
    Airport_Data, geometry=gpd.points_from_xy(Airport_Data.Longitude, Airport_Data.Latitude))
Airport_Geo_Data=Airport_Data[['City','geometry']].drop_duplicates(subset=['City']).set_index('City')

and agregate data for monthly flights:

In [ ]:
Airport_Data['YearMonth']=Airport_Data.Date-pd.offsets.MonthBegin(1)
TimeSer_Airport_Data2=Airport_Data.groupby(['YearMonth','City']).PercentOfBaseline.mean().reset_index()
TimeSer_Airport_Data_wide=TimeSer_Airport_Data2.pivot(index='YearMonth',columns='City',values='PercentOfBaseline')

In [ ]:
TimeSer_Airport_Data_wide

Lets also calculate average baselines for each city:

In [ ]:
Airport_Avgflight_Data=Airport_Data.groupby(['City']).PercentOfBaseline.mean().reset_index().set_index('City')
Airport_Geo_Data=Airport_Geo_Data.join(Airport_Avgflight_Data).reset_index()

## Visualization

Lets create a base map for USA cities and add average flight baselines as color:

In [ ]:
contiguous_usa = gpd.read_file(gplt.datasets.get_path('contiguous_usa'))
ax = gplt.polyplot(contiguous_usa, projection=gcrs.AlbersEqualArea())
gplt.pointplot(Airport_Geo_Data, ax=ax, hue='PercentOfBaseline',cmap='Greens',legend=True)

This does not show how COVID19 has effected flight baselines. We need to see the effect over time. Trying to visualize using line plots will be too cluterred. However a heatmap can tell a better story.

In [ ]:
sns.heatmap(TimeSer_Airport_Data_wide)